In [ ]:
from pynput import mouse

def on_scroll(x, y, dx, dy):
    print(f'Scrolled at ({x}, {y}) with delta ({dx}, {dy})')

with mouse.Listener(on_scroll=on_scroll) as listener:
    listener.join()

In [2]:
!pip install pywin32
!pip install pyWinhook

In [5]:
import ctypes
#mport pywin32con
from win32api import GetMessage, TranslateMessage, DispatchMessage
from win32gui import WNDCLASS, CreateWindow, DefWindowProc, RegisterClass, PumpMessages
from win32con import WM_MOUSEWHEEL, WM_DESTROY, CW_USEDEFAULT


# Define the window procedure to handle Windows messages
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == WM_MOUSEWHEEL:
        # The high-order word of wParam contains the scroll delta
        scroll_delta = ctypes.c_short(wparam >> 16).value
        print(f'Scroll event detected! Delta: {scroll_delta}')
    elif msg == WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    else:
        return DefWindowProc(hwnd, msg, wparam, lparam)
    return 0

# Register the window class
wc = WNDCLASS()
wc.lpfnWndProc = wnd_proc  # The window procedure to handle events
wc.lpszClassName = 'ScrollCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)
RegisterClass(wc)

# Create a window to listen for events
hwnd = CreateWindow(wc.lpszClassName, 'Scroll Capture', 0, CW_USEDEFAULT, CW_USEDEFAULT, CW_USEDEFAULT, CW_USEDEFAULT, 0, 0, wc.hInstance, None)

# Message loop to keep the window alive and capture scroll events
try:
    PumpMessages()
except KeyboardInterrupt:
    print("Exited.")


ImportError: cannot import name 'GetMessage' from 'win32api' (c:\Users\ezamo\miniconda3\envs\x\lib\site-packages\win32\win32api.pyd)

In [ ]:
import ctypes
import win32api
import win32con
import win32gui
from win32gui import WNDCLASS, CreateWindow, DefWindowProc, RegisterClass, PumpMessages
from win32con import WM_MOUSEWHEEL, WM_DESTROY, CW_USEDEFAULT

# Define the window procedure to handle Windows messages
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == WM_MOUSEWHEEL:
        # The high-order word of wParam contains the scroll delta
        scroll_delta = ctypes.c_short(wparam >> 16).value
        print(f'Scroll event detected! Delta: {scroll_delta}')
    elif msg == WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    else:
        return DefWindowProc(hwnd, msg, wparam, lparam)
    return 0

# Register the window class
wc = WNDCLASS()
wc.lpfnWndProc = wnd_proc  # The window procedure to handle events
wc.lpszClassName = 'ScrollCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)
RegisterClass(wc)

# Create a window to listen for events
hwnd = CreateWindow(wc.lpszClassName, 'Scroll Capture', 0, CW_USEDEFAULT, CW_USEDEFAULT, CW_USEDEFAULT, CW_USEDEFAULT, 0, 0, wc.hInstance, None)

# Message loop to keep the window alive and capture scroll events
try:
    PumpMessages()
except KeyboardInterrupt:
    print("Exited.")


In [3]:
import ctypes
import win32api
import win32con
import win32gui
import pyWinhook as pyHook

# Callback function to handle mouse wheel events
def on_mouse_event(event):
    if event.Message == win32con.WM_MOUSEWHEEL:
        # The high-order word of wParam contains the scroll delta
        scroll_delta = ctypes.c_short(event.Wheel).value
        print(f'Scroll event detected! Delta: {scroll_delta}')
    # Return True to pass the event to other handlers (if any)
    return True

# Set the hook to monitor mouse events globally
def set_mouse_hook():
    hm = pyHook.HookManager()
    hm.MouseWheel = on_mouse_event
    hm.HookMouse()

# Message loop to keep the hook running
def run_message_loop():
    while True:
        win32gui.PumpWaitingMessages()

if __name__ == "__main__":
    set_mouse_hook()
    print("Mouse scroll hook set. Start scrolling...")
    run_message_loop()


Mouse scroll hook set. Start scrolling...
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1


KeyboardInterrupt: 

In [5]:
import ctypes
import win32con
import win32gui
import ctypes.wintypes as wintypes

# Define ULONGLONG since it's not in ctypes.wintypes
ULONGLONG = ctypes.c_ulonglong

# Constants for gesture messages
WM_GESTURE = 0x0119
GID_SCROLL = 4

# Define a structure to handle gestures
class GESTUREINFO(ctypes.Structure):
    _fields_ = [
        ("cbSize", wintypes.UINT),
        ("dwFlags", wintypes.DWORD),
        ("dwID", wintypes.DWORD),
        ("hwndTarget", wintypes.HWND),
        ("ptsLocation", wintypes.POINT),
        ("dwInstanceID", wintypes.DWORD),
        ("dwSequenceID", wintypes.DWORD),
        ("ullArguments", ULONGLONG),
        ("cbExtraArgs", wintypes.UINT),
    ]

# Helper to get gesture information
def get_gesture_info(lparam):
    gesture_info = GESTUREINFO()
    gesture_info.cbSize = ctypes.sizeof(GESTUREINFO)
    ctypes.windll.user32.GetGestureInfo(lparam, ctypes.byref(gesture_info))
    return gesture_info

# Window procedure to handle messages
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == win32con.WM_MOUSEWHEEL:
        scroll_delta = ctypes.c_short(wparam >> 16).value
        print(f"Mouse scroll detected! Delta: {scroll_delta}")
    
    elif msg == WM_GESTURE:
        gesture_info = get_gesture_info(lparam)
        if gesture_info.dwID == GID_SCROLL:
            scroll_delta = ctypes.c_int64(gesture_info.ullArguments).value
            print(f"Touchpad scroll detected! Delta: {scroll_delta}")
    
    elif msg == win32con.WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    
    return win32gui.DefWindowProc(hwnd, msg, wparam, lparam)

# Register the window class
wc = win32gui.WNDCLASS()
wc.lpfnWndProc = wnd_proc
wc.lpszClassName = 'GestureCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)
win32gui.RegisterClass(wc)

# Create a window
hwnd = win32gui.CreateWindow(wc.lpszClassName, 'Gesture and Scroll Capture', 0,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             0, 0, wc.hInstance, None)

# Message loop
def run_message_loop():
    while True:
        win32gui.PumpWaitingMessages()

if __name__ == "__main__":
    print("Gesture and scroll capture started...")
    run_message_loop()


Gesture and scroll capture started...
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1


KeyboardInterrupt: 

In [6]:
import ctypes
from ctypes import wintypes
import win32api
import win32con
import win32gui
import struct

# Constants for Raw Input
RIM_TYPEMOUSE = 0
WM_INPUT = 0x00FF

# Define RAWINPUT structure
class RAWINPUTHEADER(ctypes.Structure):
    _fields_ = [
        ("dwType", wintypes.DWORD),
        ("dwSize", wintypes.DWORD),
        ("hDevice", wintypes.HANDLE),
        ("wParam", wintypes.WPARAM)
    ]

class RAWMOUSE(ctypes.Structure):
    _fields_ = [
        ("usFlags", wintypes.USHORT),
        ("ulButtons", wintypes.ULONG),
        ("usButtonFlags", wintypes.USHORT),
        ("usButtonData", wintypes.USHORT),
        ("ulRawButtons", wintypes.ULONG),
        ("lLastX", wintypes.LONG),
        ("lLastY", wintypes.LONG),
        ("ulExtraInformation", wintypes.ULONG)
    ]

class RAWINPUT(ctypes.Structure):
    _fields_ = [
        ("header", RAWINPUTHEADER),
        ("mouse", RAWMOUSE)
    ]

# Function to handle raw input messages
def handle_raw_input(lparam):
    # Get the size of the raw input data
    size = wintypes.UINT()
    ctypes.windll.user32.GetRawInputData(lparam, win32con.RID_INPUT, None, ctypes.byref(size), ctypes.sizeof(RAWINPUTHEADER))
    
    # Allocate buffer to store raw input data
    buffer = ctypes.create_string_buffer(size.value)
    
    # Get the raw input data
    ctypes.windll.user32.GetRawInputData(lparam, win32con.RID_INPUT, buffer, ctypes.byref(size), ctypes.sizeof(RAWINPUTHEADER))
    
    # Cast buffer to RAWINPUT structure
    raw = ctypes.cast(buffer, ctypes.POINTER(RAWINPUT)).contents
    
    if raw.header.dwType == RIM_TYPEMOUSE:
        mouse_data = raw.mouse
        if mouse_data.usButtonFlags & 0x0400:  # 0x0400 corresponds to a scroll event
            scroll_delta = mouse_data.usButtonData
            print(f"Scroll detected from raw input! Delta: {scroll_delta}")

# Window procedure to handle messages
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == WM_INPUT:
        handle_raw_input(lparam)
    elif msg == win32con.WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    else:
        return win32gui.DefWindowProc(hwnd, msg, wparam, lparam)
    return 0

# Register the window class
wc = win32gui.WNDCLASS()
wc.lpfnWndProc = wnd_proc
wc.lpszClassName = 'RawInputCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)
win32gui.RegisterClass(wc)

# Create a window
hwnd = win32gui.CreateWindow(wc.lpszClassName, 'Raw Input Capture', 0,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             0, 0, wc.hInstance, None)

# Register raw input devices (mouse and touchpad)
raw_mouse = (ctypes.c_int * 1)(RIM_TYPEMOUSE)
rid = ctypes.wintypes.RAWINPUTDEVICE()
rid.usUsagePage = 0x01
rid.usUsage = 0x02  # Mouse
rid.dwFlags = 0
rid.hwndTarget = hwnd
ctypes.windll.user32.RegisterRawInputDevices(ctypes.byref(rid), 1, ctypes.sizeof(rid))

# Message loop
def run_message_loop():
    while True:
        win32gui.PumpWaitingMessages()

if __name__ == "__main__":
    print("Raw input capture started...")
    run_message_loop()


AttributeError: module 'ctypes.wintypes' has no attribute 'RAWINPUTDEVICE'

In [11]:
import ctypes
import win32con
import win32gui
import ctypes.wintypes as wintypes

# Constants for Raw Input
RIM_TYPEMOUSE = 0
WM_INPUT = 0x00FF

# Define RAWINPUTHEADER structure
class RAWINPUTHEADER(ctypes.Structure):
    _fields_ = [
        ("dwType", wintypes.DWORD),
        ("dwSize", wintypes.DWORD),
        ("hDevice", wintypes.HANDLE),
        ("wParam", wintypes.WPARAM)
    ]

# Define RAWMOUSE structure
class RAWMOUSE(ctypes.Structure):
    _fields_ = [
        ("usFlags", wintypes.USHORT),
        ("ulButtons", wintypes.ULONG),
        ("usButtonFlags", wintypes.USHORT),
        ("usButtonData", wintypes.USHORT),
        ("ulRawButtons", wintypes.ULONG),
        ("lLastX", wintypes.LONG),
        ("lLastY", wintypes.LONG),
        ("ulExtraInformation", wintypes.ULONG)
    ]

# Define RAWINPUT structure
class RAWINPUT(ctypes.Structure):
    _fields_ = [
        ("header", RAWINPUTHEADER),
        ("mouse", RAWMOUSE)
    ]

# Define RAWINPUTDEVICE structure
class RAWINPUTDEVICE(ctypes.Structure):
    _fields_ = [
        ("usUsagePage", wintypes.USHORT),
        ("usUsage", wintypes.USHORT),
        ("dwFlags", wintypes.DWORD),
        ("hwndTarget", wintypes.HWND)
    ]

# Function to handle raw input messages
def handle_raw_input(lparam):
    # Get the size of the raw input data
    size = wintypes.UINT()
    ctypes.windll.user32.GetRawInputData(lparam, win32con.RID_INPUT, None, ctypes.byref(size), ctypes.sizeof(RAWINPUTHEADER))
    
    # Allocate buffer to store raw input data
    buffer = ctypes.create_string_buffer(size.value)
    
    # Get the raw input data
    ctypes.windll.user32.GetRawInputData(lparam, win32con.RID_INPUT, buffer, ctypes.byref(size), ctypes.sizeof(RAWINPUTHEADER))
    
    # Cast buffer to RAWINPUT structure
    raw = ctypes.cast(buffer, ctypes.POINTER(RAWINPUT)).contents
    
    if raw.header.dwType == RIM_TYPEMOUSE:
        mouse_data = raw.mouse
        if mouse_data.usButtonFlags & 0x0400:  # 0x0400 corresponds to a scroll event
            scroll_delta = mouse_data.usButtonData
            print(f"Scroll detected from raw input! Delta: {scroll_delta}")

# Window procedure to handle messages
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == WM_INPUT:
        handle_raw_input(lparam)
    elif msg == win32con.WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    else:
        return win32gui.DefWindowProc(hwnd, msg, wparam, lparam)
    return 0

# Register the window class
wc = win32gui.WNDCLASS()
wc.lpfnWndProc = wnd_proc
wc.lpszClassName = 'RawInputCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)

import time

# Append a unique identifier (like timestamp) to avoid conflicts
unique_class_name = 'RawInputCaptureWindow' + str(int(time.time()))

wc.lpszClassName = unique_class_name
win32gui.RegisterClass(wc)

#win32gui.RegisterClass(wc)

# Create a window
hwnd = win32gui.CreateWindow(wc.lpszClassName, 'Raw Input Capture', 0,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             0, 0, wc.hInstance, None)

# Register raw input devices (mouse and touchpad)
rid = RAWINPUTDEVICE()
rid.usUsagePage = 0x01
rid.usUsage = 0x02  # Mouse
rid.dwFlags = 0
rid.hwndTarget = hwnd
ctypes.windll.user32.RegisterRawInputDevices(ctypes.byref(rid), 1, ctypes.sizeof(rid))

# Message loop
def run_message_loop():
    while True:
        win32gui.PumpWaitingMessages()

if __name__ == "__main__":
    print("Raw input capture started...")
    run_message_loop()


Raw input capture started...
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1


KeyboardInterrupt: 

In [8]:
import ctypes
import win32con
import win32gui
import ctypes.wintypes as wintypes

# Constants for Raw Input
RIM_TYPEMOUSE = 0
WM_INPUT = 0x00FF

# Define RAWINPUTHEADER structure
class RAWINPUTHEADER(ctypes.Structure):
    _fields_ = [
        ("dwType", wintypes.DWORD),
        ("dwSize", wintypes.DWORD),
        ("hDevice", wintypes.HANDLE),
        ("wParam", wintypes.WPARAM)
    ]

# Define RAWMOUSE structure
class RAWMOUSE(ctypes.Structure):
    _fields_ = [
        ("usFlags", wintypes.USHORT),
        ("ulButtons", wintypes.ULONG),
        ("usButtonFlags", wintypes.USHORT),
        ("usButtonData", wintypes.USHORT),
        ("ulRawButtons", wintypes.ULONG),
        ("lLastX", wintypes.LONG),
        ("lLastY", wintypes.LONG),
        ("ulExtraInformation", wintypes.ULONG)
    ]

# Define RAWINPUT structure
class RAWINPUT(ctypes.Structure):
    _fields_ = [
        ("header", RAWINPUTHEADER),
        ("mouse", RAWMOUSE)
    ]

# Define RAWINPUTDEVICE structure
class RAWINPUTDEVICE(ctypes.Structure):
    _fields_ = [
        ("usUsagePage", wintypes.USHORT),
        ("usUsage", wintypes.USHORT),
        ("dwFlags", wintypes.DWORD),
        ("hwndTarget", wintypes.HWND)
    ]

# Function to handle raw input messages
def handle_raw_input(lparam):
    size = wintypes.UINT()
    ctypes.windll.user32.GetRawInputData(lparam, win32con.RID_INPUT, None, ctypes.byref(size), ctypes.sizeof(RAWINPUTHEADER))
    buffer = ctypes.create_string_buffer(size.value)
    ctypes.windll.user32.GetRawInputData(lparam, win32con.RID_INPUT, buffer, ctypes.byref(size), ctypes.sizeof(RAWINPUTHEADER))
    raw = ctypes.cast(buffer, ctypes.POINTER(RAWINPUT)).contents

    if raw.header.dwType == RIM_TYPEMOUSE:
        mouse_data = raw.mouse
        if mouse_data.usButtonFlags & 0x0400:  # Scroll event
            scroll_delta = mouse_data.usButtonData
            print(f"Scroll detected from raw input! Delta: {scroll_delta}")

# Window procedure
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == WM_INPUT:
        handle_raw_input(lparam)
    elif msg == win32con.WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    else:
        return win32gui.DefWindowProc(hwnd, msg, wparam, lparam)
    return 0

# Check if the class already exists, if not register it
wc = win32gui.WNDCLASS()
wc.lpfnWndProc = wnd_proc
wc.lpszClassName = 'RawInputCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)

try:
    win32gui.RegisterClass(wc)
except Exception as e:
    if 'Class already exists' in str(e):
        pass  # Class is already registered, proceed
    else:
        raise  # Other errors should be handled

# Create a window
hwnd = win32gui.CreateWindow(wc.lpszClassName, 'Raw Input Capture', 0,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             0, 0, wc.hInstance, None)

# Register raw input devices (mouse and touchpad)
rid = RAWINPUTDEVICE()
rid.usUsagePage = 0x01
rid.usUsage = 0x02  # Mouse
rid.dwFlags = 0
rid.hwndTarget = hwnd
ctypes.windll.user32.RegisterRawInputDevices(ctypes.byref(rid), 1, ctypes.sizeof(rid))

# Message loop
def run_message_loop():
    while True:
        win32gui.PumpWaitingMessages()

if __name__ == "__main__":
    print("Raw input capture started...")
    run_message_loop()


Raw input capture started...
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: -1


KeyboardInterrupt: 

In [10]:
import ctypes
import win32con
import win32gui

# Constants for pointer messages
WM_POINTERWHEEL = 0x024E
WM_DESTROY = 0x0002

# Window procedure to handle messages
def wnd_proc(hwnd, msg, wparam, lparam):
    if msg == WM_POINTERWHEEL:
        # Extract scroll delta from the high-order word of wParam
        scroll_delta = ctypes.c_short(wparam >> 16).value
        print(f"Touchpad scroll detected! Delta: {scroll_delta}")
    
    elif msg == WM_DESTROY:
        ctypes.windll.user32.PostQuitMessage(0)
    else:
        return win32gui.DefWindowProc(hwnd, msg, wparam, lparam)
    return 0

# Register the window class
wc = win32gui.WNDCLASS()
wc.lpfnWndProc = wnd_proc
wc.lpszClassName = 'PointerInputCaptureWindow'
wc.hInstance = ctypes.windll.kernel32.GetModuleHandleW(None)

try:
    win32gui.RegisterClass(wc)
except Exception as e:
    if 'Class already exists' in str(e):
        pass  # Class is already registered, proceed
    else:
        raise  # Other errors should be handled

# Create a window to capture pointer events
hwnd = win32gui.CreateWindow(wc.lpszClassName, 'Pointer Input Capture', 0,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT,
                             0, 0, wc.hInstance, None)

# Message loop
def run_message_loop():
    while True:
        win32gui.PumpWaitingMessages()

if __name__ == "__main__":
    print("Pointer input capture started...")
    run_message_loop()


Pointer input capture started...
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: -1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1
Scroll event detected! Delta: 1


KeyboardInterrupt: 